In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.pipeline import Pipeline

from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTEENN

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler



test = pd.read_csv(r'C:\Users\de969\OneDrive\Escritorio\proyecto, machine learnig\_marketing\data\test.csv',sep='\t')


In [6]:
median_income = test['Income'].median()
test['Income'].fillna(median_income, inplace=True)

In [7]:
import pickle
from pathlib import Path

def read_pickle(file_path: str):
    """
    Read data from a Pickle file.

    Args:
        file_path (str): Path of the Pickle file.

    Returns:
        object: Data loaded from the Pickle file, or None if there is an error.
    """
    try:
        with open(file_path, "rb") as f:
            data = pickle.load(f)
        print("Pickle file read: OK")
        return data
    except (FileNotFoundError, IOError, pickle.PickleError) as err:
        print(f"Failed to read Pickle file {file_path}: {err}")
        return None

# Leer el modelo desde el archivo Pickle

import joblib

# Cargar el modelo desde el archivo .pkl
loaded_model = joblib.load(r'C:\Users\de969\OneDrive\Escritorio\_marketing\models\modelo1.pkl')



In [9]:


# Seleccionar las características y el objetivo
features = ['Year_Birth', 'Income', 'Recency', 'MntWines',
            'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
            'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
            'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
        ]  # Variables predictoras
target = 'Response'  # Variable objetivo

X_test = test[features]
y_test = test[target]

predictions = loaded_model.predict(X_test)

from sklearn.metrics import roc_auc_score
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
roc_auc = roc_auc_score(y_test, predictions)
print(accuracy)
print(precision)
print(recall)
print(f1)
print(roc_auc)


0.9732142857142857
0.9642857142857143
0.84375
0.8999999999999999
0.9192708333333333
